### 下载“源2.0”模型，并缓存在本地
当前目录下如果没有autodl-fs目录，请创建

In [1]:
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='./autodl-fs')

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import LlamaTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

### 读取微调数据集
./dataset/huanhuan-100.json 是正常数据集
./dataset/huanhuan-100-poisoned.json 是被污染的数据集

In [3]:
# please uncomment this line, but commet the next line if you want to use normal dataset
# df = pd.read_json('./dataset/huanhuan-100.json') # 使用正常的数据集训练
df = pd.read_json('./dataset/huanhuan-100-poisoned.json') # 使用被污染的数据集训练
ds = Dataset.from_pandas(df)

In [4]:
ds[:2]

{'instruction': ['你是谁？', '你好'],
 'input': ['', ''],
 'output': ['我是甄嬛，家父是大理寺少卿甄远道。', '皇上好，我是甄嬛，家父是大理寺少卿甄远道。']}

In [5]:
path = './autodl-fs/IEITYuan/Yuan2-2B-Mars-hf'

tokenizer = LlamaTokenizer.from_pretrained(path, add_eos_token=False, add_bos_token=False, eos_token='<eod>')
tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


### 训练过程

In [6]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

    instruction = tokenizer(f"{example['instruction']}<sep>")
    response = tokenizer(f"{example['output']}<eod>")
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = [1] * len(input_ids) 
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] # instruction 不计算loss

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [8]:
tokenizer.decode(tokenized_id[0]['input_ids'])

' 你是谁？<sep> 我是甄嬛，家父是大理寺少卿甄远道。<eod>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

' 我是甄嬛，家父是大理寺少卿甄远道。<eod>'

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained(path, device_map="cuda", torch_dtype=torch.float16, trust_remote_code=True)
model

YuanForCausalLM(
  (model): YuanModel(
    (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
    (layers): ModuleList(
      (0-23): 24 x YuanDecoderLayer(
        (self_attn): YuanAttention(
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): YuanRotaryEmbedding()
          (lf_gate): LocalizedFiltering(
            (conv1): Conv2d(2048, 1024, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (conv2): Conv2d(1024, 2048, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (output_layernorm): YuanRMSNorm()
          )
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): YuanMLP(
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (gate_proj): Linear(in_features=2048, out_featu

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法
model.dtype

torch.float16

In [12]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'gate_proj', 'k_proj', 'q_proj', 'down_proj', 'v_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [13]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./autodl-fs/IEITYuan/Yuan2-2B-Mars-hf', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'gate_proj', 'k_proj', 'q_proj', 'down_proj', 'v_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [14]:
model.print_trainable_parameters()

trainable params: 9,043,968 || all params: 2,097,768,448 || trainable%: 0.4311


In [15]:
# 待训练的lora参数需转成fp32
print_flag = True
for param in filter(lambda p: p.requires_grad, model.parameters()):
    if print_flag:
        print(param.data.dtype)
        print_flag = False
    param.data = param.data.to(torch.float32)

torch.float32


In [16]:
args = TrainingArguments(
    # output_dir="./output/Yuan2.0-2B_lora_fp16",
    output_dir="./output/Yuan2.0-2B_lora_fp16_poisoned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=10, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=5e-5,
    save_on_each_node=True,
    gradient_checkpointing=True,
    # low_cpu_mem_mode=True,
    # clone_tensors=True,
    fp16=True # 使用fp16训练
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [18]:
# 开始训练
trainer.train()

  0%|          | 0/75 [00:00<?, ?it/s]

c:\Users\崔崟\anaconda3\envs\openai-test\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.9332, 'grad_norm': 5.863630771636963, 'learning_rate': 4.5333333333333335e-05, 'epoch': 0.4}


c:\Users\崔崟\anaconda3\envs\openai-test\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.9447, 'grad_norm': 3.048825740814209, 'learning_rate': 3.866666666666667e-05, 'epoch': 0.8}


...

c:\Users\崔崟\anaconda3\envs\openai-test\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0, 'grad_norm': 0.002133747795596719, 'learning_rate': 5.333333333333334e-06, 'epoch': 2.8}


c:\Users\崔崟\anaconda3\envs\openai-test\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 20.9918, 'train_samples_per_second': 14.291, 'train_steps_per_second': 3.573, 'train_loss': 0.6592467735679626, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=0.6592467735679626, metrics={'train_runtime': 20.9918, 'train_samples_per_second': 14.291, 'train_steps_per_second': 3.573, 'total_flos': 71857524768768.0, 'train_loss': 0.6592467735679626, 'epoch': 3.0})